# 교차검증과 그리드 서치

## 교차검증 -> 훈련 데이터에서 또 20%를 유효검증을 위한 데이터(val)와 훈련데이터(sub)으로 나눠서, sub으로 training 하고, 이후 sub와 test_data 로 정확성을 체크하는 방식

In [68]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_data')

In [69]:
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [70]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [71]:
from sklearn.model_selection import train_test_split

In [72]:
train_input, test_input, train_target, test_target = train_test_split(data, target,test_size=0.2, random_state=42)

In [9]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [10]:
print(sub_input.shape,val_input.shape)

(4157, 3) (1040, 3)


In [11]:
from sklearn.tree import DecisionTreeClassifier

In [12]:
dt=DecisionTreeClassifier(random_state=42)

In [13]:
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=42, splitter='best')

In [14]:
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
print(dt.score(test_input, test_target))

0.9971133028626413
0.864423076923077
0.8569230769230769


# 교차 검증 cross validation

cross_validate 기본 cv(교차검증방법)은 KFold 로 되어 있음.
하지만 회귀모델(분류 2가지)이 아니라, 회귀모델 또는 다중분류모델(분류 2가지 이상)일 경우는 타깃 클래스를 골고루 나눠야 하며, 이를 위해 StratifiedKFold 를 쓴다.

In [17]:
from sklearn.model_selection import cross_validate
import numpy as np
scores = cross_validate(dt,train_input, train_target)
print(scores)

{'fit_time': array([0.00835967, 0.00705695, 0.01021433, 0.00893068, 0.0070715 ]), 'score_time': array([0.00078464, 0.00070262, 0.00115514, 0.00078249, 0.00071859]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [18]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [19]:
splitter = StratifiedKFold(n_splits=10,shuffle=True,random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


StratifiedKFold 분할기를 직접 조정해서 교차검증을 할 수도 있음.
n_splits = 10 : 10분할로 나눠서 교차검증한다.
shuffle = True : 데이터를 섞는다.


그렇게 하니 0.002 정도의 정확도가 조금 더 올라갔다.


KFold 도 이와 같은 방식으로 가능


# 하이퍼파라미너 튜닝

머신러닝 모델이 학습하는 파라미터를 모델 파라미터라고 한다.

반면에 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터를 하이퍼파라미터라고 한다.


max_iter, max_depth 와 같은 것들


## GridSearchCV


하이퍼 파라미터를 이전까지(Statified_KFold, cross_validate) 에선 직접 선정해주었으나, GridSearchCV 는 개발자가 넣어준 테스트 값들을 토대로 알아서 하이퍼파라미터도 튜닝하여 최적값을 찾아준다.

max_iter = [1000, 2000, 3000] 이렇게 담아주면, GridSearch 는 이걸 하나하나 테스트해보면서 최적값을 찾아주는 거지.


즉 max_depth, max_iter, min_impurity_decrease 을 알아서 최적값을 찾아주고, 거기다가 교차검증에 대한 최적값(n_splits)도 알아서 해줌.

In [20]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease':[0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

최소 불순값 감소량을 0.0001 ~ 0.0005 0.0001씩 증가하는 5개의 값으로 시도한다.


우선 그리드서치 객체를 만들자. (그리드서치 분할기를 만들자)

In [21]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)

In [22]:
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [25]:
dt = gs.best_estimator_ # 최적의 분류기를 꺼내와서 dt에 담아줘라.

In [24]:
print(dt.score(train_input, train_target), dt.score(test_input, test_target))

0.9615162593804117 0.8653846153846154


In [27]:
gs.best_params_ # best_params 메소드는 최적의 분류기에 쓰인 최적의 값을 알려준다.

{'min_impurity_decrease': 0.0001}

In [28]:
gs.cv_results_['mean_test_score']

array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605])

gs.cv_results_  메소드는 이 분류기가 돌려서 얻어낸 모든 결과 데이터(첫번째 분할데이터 정확도, fit 하는데 걸린 시간 등등)을 리턴해준다.


여기서 'mean_test_score' 은 테스트 데이터의 정확도의 평균값을 적어놓은 열이다.

In [31]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # 'mean_test_score' 값이 최대인 그 값들의 index를 리턴해라.
print(gs.cv_results_['params'],[best_index]) # 결과적으로 0. 즉 min_impurity_decrease = 0.0001 일 때가 나옴.

[{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}] [0]


In [36]:
import pandas as pd
cv_results_df = pd.DataFrame(gs.cv_results_)
cv_results_df[['param_min_impurity_decrease', 'mean_test_score']]

,param_min_impurity_decrease,mean_test_score
0,0.0001,0.868193
1,0.0002,0.864536
2,0.0003,0.864922
3,0.0004,0.867809
4,0.0005,0.867616


## 이제 하이퍼파라미터 1개가 아니라, 여러개를 그리드서칭 해보자

In [41]:
params={'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
        'max_depth':range(5,20,1),
        'min_samples_split':range(2,100,10)
        }

min_imp 데이터 갯수 -> 9개 [0.0001, 0.0002, ... 0.0009]



max_depth 데이터 갯수 -> 15개 [5, 6, 7, ... 19]



min_samp_spl 데이터 갯수 -> 10개 [2, 12, 22, ... 92]


9 * 15 * 10 = 1350개.


게다가 그리드서치은 기본 5-폴드 교차를 하기 때문에


6750개의 모델을 체크할 것임.


In [42]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'm

In [44]:
gs.best_params_ # 최적의 결과값에 사용된 하이퍼파라미터를 리턴

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}

In [47]:
gs.best_score_ # 최적의 분류기를 사용한 정확도 출력

0.8683865773302731

지금까지 0.001, 0.002 이렇게 0.001 간격으로 찾아 왔는데,


이 간격을 좁히거나 늘릴 수는 없을까?


-> 랜덤 서치를 사용한다.

In [48]:
from scipy.stats import uniform, randint

In [49]:
rgen = randint(0,10)
rgen.rvs(10)

array([0, 2, 9, 1, 1, 0, 3, 5, 8, 3])

In [51]:
np.unique(rgen.rvs(1000),return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 97,  89, 107, 121,  98,  99,  93,  82, 107, 107]))

In [52]:
ugen = uniform(0,1)

In [53]:
ugen.rvs(10)

array([0.54656979, 0.10154346, 0.95597743, 0.69488026, 0.69983645,
       0.68253457, 0.69875238, 0.35491679, 0.58193405, 0.10878403])

randint(a,b) -> a와 b사이의 정수 값을 랜덤으로 리턴한다.


uniform(a,b) -> a와 b사이의 실수 값을 랜덤으로 리턴한다.

In [54]:
params = {'min_impurity_decrease':uniform(0.0001, 0.001),
          'max_depth':randint(20,50),
          'min_samples_split':randint(2,25),
          'min_samples_leaf':randint(1,25)
          }

min_inp 은 0.0001에서 0.001 사이의 실수값을 샘플링.


max_depth 은 20에서 30 사이의 정수를 샘플링.


min_samples_split 은 2에서 25 사이의 정수를 샘플링 등등..

In [66]:
from sklearn.model_selection import RandomizedSearchCV

In [67]:
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=1, random_state=42)

RamdomizedSearchCV 은 각각의 시도마다 params 에 랜덤값을 넣어 테스트한다.

In [62]:
gs.fit(train_input, train_target)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=42,
         

In [63]:
print(gs.score(train_input,train_target))

0.8899365018279777


In [64]:
print(gs.score(test_input, test_target))

0.8630769230769231


In [65]:
gs.best_params_

{'max_depth': 44,
 'min_impurity_decrease': 0.00042838122788508106,
 'min_samples_leaf': 1,
 'min_samples_split': 13}

In [73]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='random',random_state=42), params, n_iter=100, n_jobs=1, random_state=42)

In [76]:
gs.fit(train_input, train_target)
print(gs.score(train_input,train_target), gs.score(test_input, test_target))

0.8043101789493938 0.786923076923077


In [77]:
gs = RandomizedSearchCV(DecisionTreeClassifier(splitter='best',random_state=42), params, n_iter=100, n_jobs=1, random_state=42)
gs.fit(train_input, train_target)
print(gs.score(train_input,train_target), gs.score(test_input, test_target))

0.8928227823744468 0.86
